Isthmus / Mitchellpkt

March 2020

## Import libraries

In [9]:
import numpy as np
import isthmuslib as isli
import matplotlib as pyplot 
import random as random

## Parameters

In [10]:
fractional_difficulty = 0.999

## Mimic a PoW iteration

In [11]:
def hash_attempt_boolean(fractional_difficulty):
    random_num = random.random()
    if random_num >= fractional_difficulty:
        return True
    else:
        return False

## Define worker

In [16]:
class Worker:
    def __init__(self, hashes_per_second=100, search_strategy = 'increment_from_0', nonce_range = 2**32):
        self.hashes_per_second = hashes_per_second
        self.found_block_yet = False
        self.search_strategy = search_strategy
        self.nonce_range = nonce_range
        
        # Pick initial nonce
        if search_strategy == 'increment_from_0':
            self.nonce = 0
        elif search_strategy == 'increment_from_halfway':
            self.nonce = np.floor(nonce_range / 2)
        elif search_strategy == 'decrement_from_top':
            self.nonce = nonce_range
        elif search_strategy == 'uniform':
            self.nonce = int(np.floor(random.random()*self.nonce_range))

    def reset(self):
        self.nonce = 0
        self.found_block_yet = False
        
        # Pick reset nonce
        if self.search_strategy == 'increment_from_0':
            self.nonce = 0
        elif self.search_strategy == 'increment_from_halfway':
            self.nonce = np.floor(self.nonce_range / 2)
        elif self.search_strategy == 'decrement_from_top':
            self.nonce = np.floor(self.nonce_range)
        elif self.search_strategy == 'uniform':
            self.nonce = int(np.floor(random.random()*self.nonce_range))

    def single_hash(self, fractional_difficulty, verbose = False):
        
        # Make an attempt
        attempt_result = hash_attempt_boolean(fractional_difficulty)
        if attempt_result:
            self.found_block_yet = True
            
        # Update the nonce
        if (self.search_strategy == 'increment_from_0') or (self.search_strategy == 'increment_from_halfway'):
            self.nonce += 1
        elif self.search_strategy == 'decrement_from_top':
            self.nonce -= 1
        elif self.search_strategy == 'uniform':
            self.nonce = int(np.floor(random.random()*self.nonce_range))
        else:
            return NameError('Search strategy not recognized')
            
        return attempt_result
    
    def hash_n_times(self, number_of_hashes, fractional_difficulty, verbose = False):
        iteration = 0
        while (self.found_block_yet == False)& (iteration <= number_of_hashes):
            self.single_hash(fractional_difficulty)
            if verbose:
                print('--------------')
                print('Nonce: ' + str (self.nonce))
                print('Step iteration: ' + str(iteration) + ' of ' + str(number_of_hashes))
                print('Successful? ' + str(self.found_block_yet))
            iteration += 1
        return self.found_block_yet
            
    def hash_n_seconds(self, number_of_seconds, fractional_difficulty, verbose = False):
        number_of_hashes = number_of_seconds * self.hashes_per_second
        self.hash_n_times(number_of_hashes, fractional_difficulty, verbose = verbose)
        return self.found_block_yet

    def print(self):
        print('---------------')
        print('Worker hashrate: ' + str(self.hashes_per_second) + ' H/s')
        print('Search strategy: '+ self.search_strategy)
        print('Nonce range: ' + str(self.nonce_range))
        print('Current nonce: ' + str(self.nonce))
        print('Found block yet: ' + str(self.found_block_yet))

## Test a single simulation

In [33]:
W = Worker(search_strategy='increment_from_halfway')

In [34]:
W.print()

---------------
Worker hashrate: 100 H/s
Search strategy: increment_from_halfway
Nonce range: 4294967296
Current nonce: 2147483648.0
Found block yet: False


In [35]:
W.hash_n_seconds(10, 0.9, verbose=True)

--------------
Nonce: 2147483649.0
Step iteration: 0 of 1000
Successful? False
--------------
Nonce: 2147483650.0
Step iteration: 1 of 1000
Successful? False
--------------
Nonce: 2147483651.0
Step iteration: 2 of 1000
Successful? False
--------------
Nonce: 2147483652.0
Step iteration: 3 of 1000
Successful? False
--------------
Nonce: 2147483653.0
Step iteration: 4 of 1000
Successful? False
--------------
Nonce: 2147483654.0
Step iteration: 5 of 1000
Successful? False
--------------
Nonce: 2147483655.0
Step iteration: 6 of 1000
Successful? False
--------------
Nonce: 2147483656.0
Step iteration: 7 of 1000
Successful? False
--------------
Nonce: 2147483657.0
Step iteration: 8 of 1000
Successful? False
--------------
Nonce: 2147483658.0
Step iteration: 9 of 1000
Successful? False
--------------
Nonce: 2147483659.0
Step iteration: 10 of 1000
Successful? False
--------------
Nonce: 2147483660.0
Step iteration: 11 of 1000
Successful? False
--------------
Nonce: 2147483661.0
Step iteration:

True